In [2]:
from openai import OpenAI

[MYCODE] .env 파일 로드 
opeanAI 환경 설정을 .env 파일로 업로드 해서 안전하게 보관하였습니다

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

[MYCODE] 모델 설정 MLLM 사용했습니다.

few-shot 적용 
추론 방법을 명시하여 LLM이 좀더 추론을 쉽게 하기위해 메시지 작성했습니다.


[FEEDBACK]
OpenAI 대화형이 아닌 completions 를 이용했는데 앞으로 어떤 식으로 점진적으로 발전 시킬 수 있을지 기대가됩니다.

아래 방법은 few-shot을 생각하고 만들었습니다. 
이미지에 tag를 주어서 해당 태그를 이해 시키려고 했습니다.
해당 이미지를 이해하고 있는지 마지막 질문을 했을 때 이미지에 대해 답변하는 것을 확인 했지만 
실제 **최종답변** 에 대해서 그렇게 생각하고 답했을지 의문이기는 합니다. 

Q. 아래 방법이 잘못된 구현 방법이라면 조금 구체적으로 제가 참고할만 한 자료를 주시면 감사할 것 같습니다.     
Q. 이번 실습을 통해 최종적으로 이미지 상담 모델을 만들어보려고 하는데 추천할만한 모델이 있을까요 ? 

In [5]:
llm = OpenAI()

In [35]:
me_url = "https://t1.daumcdn.net/thumb/R1280x0/?fname=http://t1.daumcdn.net/brunch/service/user/AhN/image/t86QUEP5gznfgKYi9KLr60RtOBk.jpg"
item_url = "https://image-cdn.trenbe.com/product-images/1732811889714_70d413f684b3a0ccd0f38f25d83072f6_0.jpeg"
sub_item_url = "https://thumbnail10.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/ebd7/2ee15ae9f8067bcef856defc02b8e9814ca6b37af7d47e593e4306515509.jpg"

prompt = """
당신은 유명한 패션 디자이너 AI입니다.

여러 개의 이미지가 있으며, 각 이미지에는 태그명(tag)이 부여되어 있습니다. 각 설명을 참고하여 다음 질문에 답변하세요.

**구매한 아이템**
**me**가 구매한 **sell_item1**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
**답변:** 물건: 모자, 색상: 검정색, 모양: 체크무늬

**구매한 아이템**
**me**가 구매한 **sell_item2**은 어떤 물건이며, 어떤 색과 모양을 가지고 있나요?
**답변:** 물건: 점퍼, 색상: 금색, 모양: 민무늬

**질문 1:**
**me**의 키는 몇인가요?
**답변:** 180

**질문 2:**
**me**의 머리 스타일은 무엇인가요?
**답변:** 짧다.

**질문 3:**
**me**의 손은 무엇이 있나요?
**답변:** 팔찌

**질문 4:**
**me**는 상체에 어떤 옷을 입었나요?
**답변:** 검정색 티셔츠

**질문 5:**
**me**는 하체는 어떤 옷을 입었나요?
**답변:** 검정색 청바지

**질문 6:**
**me**의 성별은 무엇인가요?
**답변:** 남자

**중간 요약:**
키가 180 정도 되는 남자는 머리가 짧고, 손에는 팔찌를 착용했으며, 상체는 검정색 티셔츠, 하체는 검정색 청바지를 입고 있습니다.

**me** 이미지에 어울리는 패션 아이템 세가지를 **구매한 아이템** 참고해서 추천 해주세요

# 최종답변
{
  "recommended_items": [
    {
      "item": "슬렉스",
      "color": "검정색",
      "style": "심플한 디자인"
    },
    {
      "item": "가방",
      "color": "갈색",
      "style": "캔버스 재질의 쇼퍼백"
    },
    {
      "item": "목걸이",
      "color": "금색",
      "style": "체인"
    }
  ]
  "recommended_items_description" : "현재 입은 옷과 **구매한 아이템** 참고하여 위 상품을 추천합니다. 가방은 좀더 개성을 보여줄 것 같네요. 머리와 잘 어울리는 패션이될 것 같네요",
}

//----------------------few-shot end

 위 방법으로 추론해서 **sell_item** 이미지를 참고하여 **me** 이미지에 어울리는 상품 3 가지를 추천해주세요

# 최종답변
"""

response = llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                
                {"type": "text", "text": prompt},                    
                {"type": "image_url", "image_url" :{"url": me_url, "tag" : "me"}}, 
                {"type": "image_url", "image_url" :{"url": item_url, "tag" : "sell_item"}}, 
                {"type": "image_url", "image_url" :{"url": sub_item_url, "tag" : "sell_item"}}, 
            ],
        }
    ],
    max_tokens=300,
    
)

In [7]:
import json

In [34]:
print(response.choices[0].message.content)

# 최종답변
{
  "recommended_items": [
    {
      "item": "구두",
      "color": "갈색",
      "style": "가죽 소재로 클래식한 디자인"
    },
    {
      "item": "시계",
      "color": "은색",
      "style": "심플한 메탈 팔찌 시계"
    },
    {
      "item": "셔츠",
      "color": "흰색",
      "style": "네이비 블레이저와 잘 어울리는 기본 디자인"
    }
  ],
  "recommended_items_description": "현재 입은 옷과 **구매한 아이템** 모자와 점퍼를 참고하여 위 상품을 추천합니다. 구두는 전체적인 격식을 더해줄 것이며, 시계는 실용적이면서도 스타일리시할 것입니다. 또한, 흰색 셔트는 블레이저와 조화를 이루어 멋스러운 룩을 완성해 줄 것입니다."
}


In [36]:
content = response.choices[0].message.content
json.loads(content.replace("# 최종답변", ""))

{'recommended_items': [{'item': '블랙 슬랙스', 'color': '검정색', 'style': '슬림핏'},
  {'item': '가벼운 자켓', 'color': '회색', 'style': '스포츠 캐주얼'},
  {'item': '가죽 팔찌', 'color': '갈색', 'style': '심플한 디자인'}],
 'recommended_items_description': '현재 입은 옷과 **구매한 아이템**을 참고하여 위 상품을 추천합니다. 블랙 슬랙스는 특정한 룩을 완성해 줄 것이며, 가벼운 자켓은 캐주얼하면서도 스타일을 더할 수 있습니다. 가죽 팔찌는 전체적인 포인트가 될 것 같습니다.'}